### Парсинг телефонов с Авито

In [18]:
import sys
sys.path.append('../..')


In [19]:
import os
import re
import time

import pandas as pd
from bs4 import BeautifulSoup
from requests import Session, session
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager

from src.db import ProductTable, SellerTable, SessionLocal, engine
from src.parser.models import Product, Seller
from src.parser.parser import parse_avito_page
from src.parser.tools import get_ad_urls, get_photos

### Константы

In [20]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 100
product_columns = [
'title', 'price', 'characteristics', 
'description', 'views', 'date',
'location', 'link', 'seller_id', 'today_views'
, 'about', 'is_sold'
]

seller_columns = [
    'seller_id', 'name', 'rating', 'reviews',
    'subscribers', 'subscriptions', 'registered', 
    'done_deals', 'active_deals', 'docs_confirmed',
    'phone_confirmed', 'response_time'
]

db_session = SessionLocal()
# списки существующих ключей для того, чтобы лишний раз 
# не добавлять объявление/ продавца в  таблицу
product_ids = db_session.query(ProductTable.link).all()
product_ids = [id[0] for id in product_ids]

seller_ids = db_session.query(SellerTable.seller_id).all()
seller_ids = [id[0] for id in seller_ids]

### Запуск сервиса

In [21]:
service = Service(GeckoDriverManager().install())
# add headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
}

# create a new session
session = Session()
session.headers.update(headers)

# dont open browser
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(service=service, options=options)

### Максимальный номер сохраненной страницы  

In [22]:
output_folder = "data8"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: None
Начнем парсинг со страницы номер: 1 (вкл)
Закончим парсинг на странице номер: 101 (не вкл)


### Парсинг

In [ ]:
for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=product_columns)
    df_seller = pd.DataFrame(columns=seller_columns)

    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            if link in product_ids:
                continue
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data, seller_data, done_deals_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                # ad_data['photo'] = get_photos(driver=driver, cnt=5)

                # валидация используя Pydantic
                product = Product.model_validate(ad_data)
                seller = Seller.model_validate(seller_data)

                product_ids.append(link)
                db_session.add(ProductTable(**product.model_dump()))
                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)
                # проверка на отсутствие продавца в таблице
                if seller_data['seller_id'] not in seller_ids:
                    seller_ids.append(seller_data['seller_id'])
                    db_session.add(SellerTable(**seller.model_dump()))
                    df_seller = pd.concat([df_seller, pd.DataFrame([seller_data])], ignore_index=True)

                    done_deals_list = [Product.model_validate(deal) for deal in done_deals_data]
                    for deal in done_deals_list:
                        db_session.add(ProductTable(**deal.model_dump()))
                    for deal in done_deals_data:
                        df_page = pd.concat([df_page, pd.DataFrame([deal])], ignore_index=True)
                # break
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue
            db_session.commit()
        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/phones_data_page_{page_num}.parquet')
        df_seller.to_parquet(f'{output_folder}/sellers_data_page_{page_num}.parquet')
        # break
    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue

db_session.close()



Начали парсинг страницы # 0
Произошла ошибка на странице 0: HTTPConnectionPool(host='localhost', port=60257): Read timed out. (read timeout=120)
Начали парсинг страницы # 1


  2%|▏         | 1/50 [00:07<06:01,  7.37s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_13_pro_128_gb_3646507026?context=H4sIAAAAAAAA_wEmANn_YToxOntzOjE6IngiO3M6MTY6IkxBdmZjMEYxOVgxNmY1YTIiO30lPZmRJgAAAA: Message: Move target (661, 895) is out of bounds of viewport dimensions (1366, 683)
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
MoveTargetOutOfBoundsError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:479:5
assertTargetInViewPort@chrome://remote/content/shared/webdriver/Actions.sys.mjs:3097:11
#assertInViewPortFromContent@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:90:17
dispatch@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1588:11



  4%|▍         | 2/50 [00:13<05:28,  6.83s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_13_pro_256_gb_3659361037?context=H4sIAAAAAAAA_wEmANn_YToxOntzOjE6IngiO3M6MTY6IkxBdmZjMEYxOVgxNmY1YTIiO30lPZmRJgAAAA: Message: Move target (660, 896) is out of bounds of viewport dimensions (1366, 683)
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
MoveTargetOutOfBoundsError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:479:5
assertTargetInViewPort@chrome://remote/content/shared/webdriver/Actions.sys.mjs:3097:11
#assertInViewPortFromContent@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:90:17
dispatch@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1588:11



  6%|▌         | 3/50 [00:49<12:55, 16.51s/it]


KeyboardInterrupt: 

In [ ]:
driver.quit()

In [ ]:
# read data from parquet files
data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet') and file.startswith('sellers'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)

# get parquet files in 
print(len(data))
data[0].head()


0


IndexError: list index out of range

In [ ]:
data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet') and file.startswith('phones'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)

# get parquet files in 
print(len(data))
data[0].head(10)

1


,title,price,characteristics,description,views,date,location,link,seller_id,today_views,about,is_sold
0,"iPhone 11 Pro, 256 ГБ",22990,"{'Встроенная память': '256 ГБ', 'Модель': 'iPh...",📲 В пpодaжe Aрplе iРhоnе 11 Prо - oригинал,769,· 24 февраля в 20:45,"Санкт-Петербург, ул. Рубинштейна, 24",https://www.avito.ru/sankt-peterburg/telefony/...,/brands/fonmart,6,"{'Корпус': 'Без дефектов', 'Состояние': 'Отлич...",False
